## Что первое пришло в голову ##

Блин, не применить tf/idf, ngram и word2vec (хотя про последнее не факт, но я ещё не придумал как)

Необычная NLP задача, по факту нужно классифицировать класс каждой буквы в слове на основании каких-то признаков.


Потенциальные признаки: векторное представление слова (циферка для каждой буквы, желательно по-умному нормализовать, потому что нормализовать по алфавиту - не очень, скорее всего нужно будет присваивать цифру в зависимости от частоты встречаемости), длина слова, число гласных/согласных, позиция слова в изначальном названии компании, внутри ли кавычек слово.

Дальше можно будет проверить разные классификаторы.

In [32]:
import numpy as np
import pandas as pd

MAX_WORD = 69

CONSONANTS = 'бвгджзйклмнпрстфхцчшщБВГДЖЗЙКЛМНПРСТФХЦЧШЩ'
VOWELS = 'аиоуэыеёюяАИОУЭЫЕЁЮЯ'
ALPHABET = 'АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдеёжзийклмнопрстуфхцчшщъыьэюя'

In [2]:
train_companies = open('kontur_srs_internship_test_task/train.txt', 'r').readlines()
train_companies = list(map(lambda s: s.strip(), train_companies))

test_companies = open('kontur_srs_internship_test_task/test.txt', 'r').readlines()
test_companies = list(map(lambda s: s.strip(), test_companies))

In [3]:
import re

def find_max_str(companies):
    max_str = ''
    for company in companies:
        for word in re.split(' "| |" |"', company):
            if(word):
                max_str = word if len(max_str) < len(word) else max_str 
    return max_str

In [4]:
find_max_str(test_companies)

'СНАБЖЕНЧЕСКО-СБЫТОВОЙ-ПЕРЕРАБАТЫВАЮЩИЙ-ОБСЛУЖИВАЮЩИЙ-ЖИВОТНОВОДЧЕСКИЙ'

In [5]:
def find_index_of_substring(string, companies):
    for i in range(len(companies)):
        if(train_companies[i].find(string) != -1):
            return i

In [6]:
find_index_of_substring(find_max_str(test_companies), test_companies)

In [7]:
find_index_of_substring(find_max_str(train_companies), train_companies)

471970

In [8]:
del train_companies[471970]

In [9]:
test_companies[326264]

'МУНИЦИПАЛЬНОЕ БЮДЖЕТНОЕ ОБРАЗОВАТЕЛЬНОЕ УЧРЕЖДЕНИЕ ДОПОЛНИТЕЛЬНОГО ОБРАЗОВАНИЯ ДЕТЕЙ "ТАШЛИНСКАЯ ДЕТСКАЯ ШКОЛА ИСКУССТВ"'

In [10]:
len(find_max_str(test_companies))

69

In [11]:
from string import ascii_lowercase
from collections import Counter

with open('kontur_srs_internship_test_task/train.txt') as f:
    print(Counter(letter for line in f 
                  for letter in line.lower() 
                  if letter in ALPHABET))

KeyboardInterrupt: 

In [12]:
def has_letters(string):
    return any(char.lower() in ALPHABET for char in string)

def has_not_letters(string):
    return any(char.lower() not in ALPHABET for char in string)

def count_vowels(string):
    return len([char for char in string if char in VOWELS])

def count_consonants(string):
    return len([char for char in string if char in CONSONANTS])

In [13]:
import nltk
from nltk import word_tokenize

def extract_features(company_name, test=False):
    
    #Длина слова, количество гласных/согласных, положение в слове и есть ли в слове что-то помимо букв
    num_of_features = 5
    
    #3 класса - полностью писменными, первая заглавная или полностью заглавными
    num_of_classes = 3

    size = 0
    for word in word_tokenize(company_name):
        if(has_letters(word)):
            size += 1
            
    features = np.zeros((size, num_of_features))
    labels = np.zeros(size)
    
    index = 0
    for word in word_tokenize(company_name):
        if(has_letters(word)):
            features[index][0] = index/size
            features[index][1] = len(word)/MAX_WORD
            features[index][2] = count_consonants(word)/len(word)
            features[index][3] = count_vowels(word)/len(word)
            features[index][4] = int(has_not_letters(word))
            if(word[0].isupper()):
                if(word[-1].isupper()):
                    labels[index] = 2
                else:
                    labels[index] = 1
            else:
                labels[index] = 0
            index += 1
    if(not test):    
        return features, labels
    else:
        return features

In [160]:
a = extract_features('Товарищество собственников жилья "Вавилова 8/26А"')

In [14]:
def generate_dataset(companies, test=False):
    features = np.array([])
    labels = np.array([])
    for company in companies:
        curr_features, curr_labels = extract_features(company)
        if(len(features)):
            features = np.concatenate((features, curr_features))
            labels = np.concatenate((labels, curr_labels))
        else:
            features = curr_features
            labels = curr_labels
    if(not test):    
        return features, labels
    else:
        return features 

In [34]:
a, b = generate_dataset(train_companies[:50000])

In [225]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from numpy import mean
from numpy import std

In [226]:
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
# cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# n_scores = cross_val_score(model, a, b, scoring='accuracy', cv=cv, n_jobs=-1)
# print('Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

In [227]:
model.fit(a, b)

/home/nikita/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(multi_class='multinomial')

In [219]:
len(train_companies)

5232476

In [220]:
test_companies

['ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ РАДИОКОМПАНИЯ "ВОСТОК"',
 'ГБУ "АЗНАКАЙЛЕС"',
 'ОАО "АЗНАКАЕВСКОЕ ПАТП"',
 'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "АЗНАКАЕВСКОЕ ПАССАЖИРСКОЕ АВТОТРАНСПОРТНОЕ ПРЕДПРИЯТИЕ-1"',
 'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "АЛНАЗ"',
 'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "МАГИСТРАЛЬ"',
 'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ЭНТЕР"',
 'ЗАКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО "АЗГЕОФИЗИКА"',
 'ГИАГИНСКАЯ РАЙОННАЯ ОБЩЕСТВЕННАЯ ОРГАНИЗАЦИЯ АДЫГЕЙСКОЙ РЕСПУБЛИКАНСКОЙ ОБЩЕСТВЕННОЙ ОРГАНИЗАЦИИ ОХОТНИКОВ И РЫБОЛОВОВ',
 'РО ПП "КОПРФ" В РА',
 'АДЫГЕЙСКАЯ РЕСПУБЛИКАНСКАЯ ОРГАНИЗАЦИЯ НЕЗАВИСИМОГО ПРОФСОЮЗА РАБОТНИКОВ ОРГАНИЗАЦИЙ ТОРГОВЛИ И УСЛУГ "ПЕРСПЕКТИВА"',
 'АКЦИОНЕРНОЕ ОБЩЕСТВО ОТКРЫТОГО ТИПА "АКЦИОНЕРНО-КОММЕРЧЕСКИЙ БАНК СОЛИД"',
 'ПЕРВИЧНАЯ ПРОФСОЮЗНАЯ ОРГАНИЗАЦИЯ ОАО "АВИАЛИНИИ АДЫГЕИ"',
 'МАЙКОПСКАЯ ГОРОДСКАЯ ОРГАНИЗАЦИЯ "ДОМ АДАПТАЦИИ РЕПАТРИАНТОВ"',
 'ОД "АДЫГЭ ХАСЭ - ЧЕРКЕССКИЙ ПАРЛАМЕНТ" РА',
 'МЕСТНАЯ ОРГАНИЗАЦИЯ ОБЩЕРОССИЙСКОЙ ОБЩЕСТВЕННОЙ О

In [16]:
def transform(company_name, labels):
    index = 0
    for word in word_tokenize(company_name):
        if(has_letters(word)):
            if(labels[index] == 0):
                print(word.lower())
            elif(labels[index] == 1):
                print(word[0] + word[1:].lower())
            else:
                print(word)
            index += 1

In [44]:
company = 'ООО РК "Восток"'
extract_features(company, test=True)

array([[0.        , 0.04347826, 0.        , 1.        , 0.        ],
       [0.33333333, 0.02898551, 1.        , 0.        , 0.        ],
       [0.66666667, 0.08695652, 0.66666667, 0.33333333, 0.        ]])

In [229]:
transform(company, model.predict(extract_features(company, test=True)))

фгаоу
ДПО
адыгейский
Цппк


In [199]:
test

array([[0.        , 0.1884058 , 0.46153846, 0.46153846, 0.        ],
       [0.1       , 0.2173913 , 0.46666667, 0.46666667, 0.        ],
       [0.2       , 0.14492754, 0.5       , 0.5       , 0.        ],
       [0.3       , 0.2173913 , 0.53333333, 0.4       , 0.        ],
       [0.4       , 0.15942029, 0.45454545, 0.45454545, 0.        ],
       [0.5       , 0.07246377, 0.6       , 0.4       , 0.        ],
       [0.6       , 0.17391304, 0.58333333, 0.33333333, 0.        ],
       [0.7       , 0.10144928, 0.57142857, 0.28571429, 0.        ],
       [0.8       , 0.07246377, 0.6       , 0.4       , 0.        ],
       [0.9       , 0.11594203, 0.75      , 0.25      , 0.        ]])

In [35]:
from catboost import CatBoostClassifier
clf = CatBoostClassifier()
clf.fit(a, b)

Learning rate set to 0.106383
0:	learn: 0.9961313	total: 83ms	remaining: 1m 22s
1:	learn: 0.9180479	total: 166ms	remaining: 1m 22s
2:	learn: 0.8554086	total: 265ms	remaining: 1m 27s
3:	learn: 0.8016490	total: 356ms	remaining: 1m 28s
4:	learn: 0.7587590	total: 452ms	remaining: 1m 29s
5:	learn: 0.7229263	total: 551ms	remaining: 1m 31s
6:	learn: 0.6915697	total: 636ms	remaining: 1m 30s
7:	learn: 0.6642231	total: 723ms	remaining: 1m 29s
8:	learn: 0.6406460	total: 816ms	remaining: 1m 29s
9:	learn: 0.6198662	total: 899ms	remaining: 1m 29s
10:	learn: 0.6020778	total: 989ms	remaining: 1m 28s
11:	learn: 0.5860692	total: 1.09s	remaining: 1m 29s
12:	learn: 0.5720652	total: 1.17s	remaining: 1m 28s
13:	learn: 0.5598649	total: 1.25s	remaining: 1m 27s
14:	learn: 0.5476841	total: 1.35s	remaining: 1m 28s
15:	learn: 0.5373765	total: 1.43s	remaining: 1m 27s
16:	learn: 0.5278632	total: 1.52s	remaining: 1m 27s
17:	learn: 0.5197207	total: 1.61s	remaining: 1m 28s
18:	learn: 0.5121160	total: 1.7s	remaining: 1

157:	learn: 0.3804425	total: 15.5s	remaining: 1m 22s
158:	learn: 0.3802752	total: 15.6s	remaining: 1m 22s
159:	learn: 0.3800087	total: 15.7s	remaining: 1m 22s
160:	learn: 0.3797833	total: 15.8s	remaining: 1m 22s
161:	learn: 0.3796399	total: 15.9s	remaining: 1m 22s
162:	learn: 0.3794118	total: 16s	remaining: 1m 22s
163:	learn: 0.3792681	total: 16.1s	remaining: 1m 22s
164:	learn: 0.3790109	total: 16.2s	remaining: 1m 21s
165:	learn: 0.3788244	total: 16.3s	remaining: 1m 21s
166:	learn: 0.3785528	total: 16.4s	remaining: 1m 21s
167:	learn: 0.3783369	total: 16.5s	remaining: 1m 21s
168:	learn: 0.3781983	total: 16.6s	remaining: 1m 21s
169:	learn: 0.3779938	total: 16.7s	remaining: 1m 21s
170:	learn: 0.3777251	total: 16.8s	remaining: 1m 21s
171:	learn: 0.3775985	total: 16.9s	remaining: 1m 21s
172:	learn: 0.3772568	total: 17s	remaining: 1m 21s
173:	learn: 0.3770924	total: 17.1s	remaining: 1m 21s
174:	learn: 0.3768227	total: 17.2s	remaining: 1m 20s
175:	learn: 0.3766013	total: 17.3s	remaining: 1m 2

313:	learn: 0.3610271	total: 30.9s	remaining: 1m 7s
314:	learn: 0.3609519	total: 31s	remaining: 1m 7s
315:	learn: 0.3608853	total: 31.1s	remaining: 1m 7s
316:	learn: 0.3608214	total: 31.2s	remaining: 1m 7s
317:	learn: 0.3607444	total: 31.3s	remaining: 1m 7s
318:	learn: 0.3606947	total: 31.3s	remaining: 1m 6s
319:	learn: 0.3606119	total: 31.4s	remaining: 1m 6s
320:	learn: 0.3605584	total: 31.5s	remaining: 1m 6s
321:	learn: 0.3604791	total: 31.6s	remaining: 1m 6s
322:	learn: 0.3603764	total: 31.7s	remaining: 1m 6s
323:	learn: 0.3603250	total: 31.8s	remaining: 1m 6s
324:	learn: 0.3602368	total: 31.9s	remaining: 1m 6s
325:	learn: 0.3602103	total: 32s	remaining: 1m 6s
326:	learn: 0.3601103	total: 32.1s	remaining: 1m 5s
327:	learn: 0.3600429	total: 32.2s	remaining: 1m 5s
328:	learn: 0.3599848	total: 32.2s	remaining: 1m 5s
329:	learn: 0.3599373	total: 32.3s	remaining: 1m 5s
330:	learn: 0.3598743	total: 32.4s	remaining: 1m 5s
331:	learn: 0.3598016	total: 32.5s	remaining: 1m 5s
332:	learn: 0.35

475:	learn: 0.3521307	total: 46.4s	remaining: 51s
476:	learn: 0.3521162	total: 46.4s	remaining: 50.9s
477:	learn: 0.3520878	total: 46.5s	remaining: 50.8s
478:	learn: 0.3520437	total: 46.6s	remaining: 50.7s
479:	learn: 0.3519744	total: 46.8s	remaining: 50.7s
480:	learn: 0.3519170	total: 46.9s	remaining: 50.6s
481:	learn: 0.3518702	total: 47s	remaining: 50.5s
482:	learn: 0.3518379	total: 47.1s	remaining: 50.4s
483:	learn: 0.3518013	total: 47.2s	remaining: 50.3s
484:	learn: 0.3517695	total: 47.3s	remaining: 50.2s
485:	learn: 0.3516655	total: 47.4s	remaining: 50.1s
486:	learn: 0.3515991	total: 47.5s	remaining: 50s
487:	learn: 0.3515708	total: 47.6s	remaining: 49.9s
488:	learn: 0.3515206	total: 47.7s	remaining: 49.8s
489:	learn: 0.3514836	total: 47.7s	remaining: 49.7s
490:	learn: 0.3514266	total: 47.8s	remaining: 49.6s
491:	learn: 0.3513788	total: 47.9s	remaining: 49.4s
492:	learn: 0.3513633	total: 48s	remaining: 49.3s
493:	learn: 0.3513247	total: 48s	remaining: 49.2s
494:	learn: 0.3512863	

635:	learn: 0.3467860	total: 1m	remaining: 34.5s
636:	learn: 0.3467604	total: 1m	remaining: 34.4s
637:	learn: 0.3467316	total: 1m	remaining: 34.3s
638:	learn: 0.3466806	total: 1m	remaining: 34.2s
639:	learn: 0.3466725	total: 1m	remaining: 34.1s
640:	learn: 0.3466414	total: 1m	remaining: 33.9s
641:	learn: 0.3466192	total: 1m	remaining: 33.8s
642:	learn: 0.3465609	total: 1m	remaining: 33.7s
643:	learn: 0.3465287	total: 1m	remaining: 33.6s
644:	learn: 0.3464930	total: 1m	remaining: 33.5s
645:	learn: 0.3464764	total: 1m 1s	remaining: 33.4s
646:	learn: 0.3464488	total: 1m 1s	remaining: 33.3s
647:	learn: 0.3464175	total: 1m 1s	remaining: 33.2s
648:	learn: 0.3464031	total: 1m 1s	remaining: 33.1s
649:	learn: 0.3463741	total: 1m 1s	remaining: 33s
650:	learn: 0.3463394	total: 1m 1s	remaining: 32.9s
651:	learn: 0.3463056	total: 1m 1s	remaining: 32.9s
652:	learn: 0.3462859	total: 1m 1s	remaining: 32.8s
653:	learn: 0.3462653	total: 1m 1s	remaining: 32.7s
654:	learn: 0.3462528	total: 1m 1s	remaining

793:	learn: 0.3430698	total: 1m 15s	remaining: 19.5s
794:	learn: 0.3430630	total: 1m 15s	remaining: 19.4s
795:	learn: 0.3430245	total: 1m 15s	remaining: 19.3s
796:	learn: 0.3430131	total: 1m 15s	remaining: 19.2s
797:	learn: 0.3430047	total: 1m 15s	remaining: 19.1s
798:	learn: 0.3429881	total: 1m 15s	remaining: 19s
799:	learn: 0.3429517	total: 1m 15s	remaining: 18.9s
800:	learn: 0.3429302	total: 1m 15s	remaining: 18.8s
801:	learn: 0.3429073	total: 1m 15s	remaining: 18.7s
802:	learn: 0.3428981	total: 1m 15s	remaining: 18.6s
803:	learn: 0.3428839	total: 1m 15s	remaining: 18.5s
804:	learn: 0.3428502	total: 1m 16s	remaining: 18.4s
805:	learn: 0.3428277	total: 1m 16s	remaining: 18.3s
806:	learn: 0.3428189	total: 1m 16s	remaining: 18.2s
807:	learn: 0.3428088	total: 1m 16s	remaining: 18.1s
808:	learn: 0.3427968	total: 1m 16s	remaining: 18s
809:	learn: 0.3427783	total: 1m 16s	remaining: 18s
810:	learn: 0.3427660	total: 1m 16s	remaining: 17.9s
811:	learn: 0.3427569	total: 1m 16s	remaining: 17.8s

949:	learn: 0.3403984	total: 1m 30s	remaining: 4.74s
950:	learn: 0.3403908	total: 1m 30s	remaining: 4.65s
951:	learn: 0.3403780	total: 1m 30s	remaining: 4.55s
952:	learn: 0.3403654	total: 1m 30s	remaining: 4.46s
953:	learn: 0.3403561	total: 1m 30s	remaining: 4.36s
954:	learn: 0.3403455	total: 1m 30s	remaining: 4.27s
955:	learn: 0.3403346	total: 1m 30s	remaining: 4.17s
956:	learn: 0.3403196	total: 1m 30s	remaining: 4.08s
957:	learn: 0.3403041	total: 1m 30s	remaining: 3.98s
958:	learn: 0.3402989	total: 1m 31s	remaining: 3.89s
959:	learn: 0.3402874	total: 1m 31s	remaining: 3.8s
960:	learn: 0.3402716	total: 1m 31s	remaining: 3.7s
961:	learn: 0.3402512	total: 1m 31s	remaining: 3.61s
962:	learn: 0.3402405	total: 1m 31s	remaining: 3.51s
963:	learn: 0.3402193	total: 1m 31s	remaining: 3.42s
964:	learn: 0.3402020	total: 1m 31s	remaining: 3.32s
965:	learn: 0.3401891	total: 1m 31s	remaining: 3.23s
966:	learn: 0.3401733	total: 1m 31s	remaining: 3.13s
967:	learn: 0.3401565	total: 1m 31s	remaining: 3

In [45]:
transform(company, clf.predict(extract_features(company, test=True)))

ООО
РК
Восток


In [30]:
extract_features('Общество с ограниченной ответственностью "Сельэнерго-2"', test=True)

array([[0.        , 0.11594203, 0.625     , 0.375     , 0.        ],
       [0.2       , 0.01449275, 1.        , 0.        , 0.        ],
       [0.4       , 0.17391304, 0.58333333, 0.41666667, 0.        ],
       [0.6       , 0.23188406, 0.625     , 0.3125    , 0.        ],
       [0.8       , 0.17391304, 0.41666667, 0.33333333, 1.        ]])

In [28]:
train_companies[:10]

['ООО РК "Восток"',
 'Общество с ограниченной ответственностью "Сельэнерго-2"',
 'Коммунальное предприятие Азнакаевского районно-производственного управления бытового обслуживания населения',
 'КП Азнакаевского РПУ БОН',
 'Общество с ограниченной ответственностью "Здоровье"',
 'Общество с ограниченной ответственностью "Рузаль"',
 'Государственное бюджетное учреждение Республики Татарстан "Азнакайлес"',
 'Открытое акционерное общество "Азтрубстрой"',
 'ОАО "Азтрубстрой"',
 'Закрытое акционерное общество "Северное сияние"']

In [33]:
len(CONSONANTS + VOWELS)

62